## Huntbook
# Tracing Covenant ShellCmd (via InstallUtil)

This interactive Kestrel huntbook is about tracing an adversary, that proxy executes code trough the InstallUtil, a command-line utility tool included in the . NET Framework.

The adversary uses it to install the "GruntHTTP.dll", a trojan malware and Covenant’s implant that is deployed to target system.


## Dataset Description

The dataset focusses on the proxy code execution via InstallUtil, a trusted Windows util.

Dataset: https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/defense_evasion/host/covenant_installutil.zip



## Adversary View

```
Upload Task: GruntHTTP.dll -> C:\ProgramData\GruntHTTP.dll
(wardog) > ShellCmd /shellcommand:"C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll"
```

Source: https://securitydatasets.com/notebooks/atomic/windows/execution/SDWIN-191027223020.html

## Tactics & Techniques

For the attack, the tactics [TA0005](https://attack.mitre.org/tactics/TA0005/), [TA0002](https://attack.mitre.org/tactics/TA0002/) and the techniques [T1218.004](https://attack.mitre.org/techniques/T1218/004/) are used.

The adversary uses Defense Evation, trying to avoid being detected and runs malicious code with Execution tactics. He uses System Binary Proxy Execution with the InstallUtil, a sub-technique of [T1218](https://attack.mitre.org/techniques/T1218/).

## Get Processes and Filter for "InstallUtil"

Typically the InstallUtil is used by developers to install and configure services or service applications. They can specify the '.exe' file of a Windows Service and install it onto the operating system. Here it is used to install malicious on the system.

In [1]:
procs = GET process from https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/defense_evasion/host/covenant_installutil.zip 
where command_line LIKE '%InstallUtil%'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
procs,process,2,147,2,230,246,2,1,153,8,45,147,147


We found records of two process entities. To examine them further, we display them.

In [3]:
DISP procs ATTR pid, x_unique_id, created, name, command_line 

sproc = procs

pid 
 x_unique_id 
 created 
 name 
 command_line 
 
 
 
 
 3776.0 
 {b34bc01c-add5-5f66-2010-000000000400} 
 2020-09-20T01:18:13.287Z 
 cmd.exe 
 "cmd.exe" /c C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll 
 
 
 
 {b34bc01c-add5-5f66-2210-000000000400} 
 2020-09-20T01:18:13.416Z 
 InstallUtil.exe 
 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll 
 
 
 
 Block Executed in 2 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 sproc 
 process 
 2 
 147 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 *Number of related records cached.

## Analyze the Process
In the next hunting step, we analyze the suspicious process we found.

In [4]:
processes = FIND process CREATED BY sproc

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
processes,process,2,290,2,466,515,4,2,313,10,89,305,305


In [5]:
DISP processes

x_unique_id,created,name,command_line,id,pid,binary_ref.name,binary_ref.hashes.'SHA-1',binary_ref.hashes.MD5,binary_ref.hashes.'SHA-256',binary_ref.id,binary_ref.parent_directory_ref.path,binary_ref.parent_directory_ref.id,parent_ref.x_unique_id,parent_ref.created,parent_ref.name,parent_ref.command_line,parent_ref.id,parent_ref.pid,parent_ref.parent_ref.x_unique_id,parent_ref.parent_ref.created,parent_ref.parent_ref.name,parent_ref.parent_ref.command_line,parent_ref.parent_ref.id,parent_ref.parent_ref.pid,parent_ref.binary_ref.name,parent_ref.binary_ref.hashes.'SHA-1',parent_ref.binary_ref.hashes.MD5,parent_ref.binary_ref.hashes.'SHA-256',parent_ref.binary_ref.id,parent_ref.binary_ref.parent_directory_ref.path,parent_ref.binary_ref.parent_directory_ref.id,parent_ref.creator_user_ref.user_id,parent_ref.creator_user_ref.id,creator_user_ref.user_id,creator_user_ref.id
{b34bc01c-add5-5f66-2110-000000000400},2020-09-20T01:18:13.294Z,conhost.exe,\??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1,process--820ee083-891f-5a3b-a668-f81b94c0ed68,None,conhost.exe,None,None,None,file--9d6a9721-9948-53ff-83f8-3865bc03c727,C:\Windows\System32,directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622,{b34bc01c-add5-5f66-2010-000000000400},2020-09-20T01:18:13.287Z,cmd.exe,"""cmd.exe"" /c C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll",process--d689999d-1ba7-5310-8af6-78edc64d9b28,3776,{b34bc01c-0266-5f65-540c-000000000400},None,GruntHTTP.exe,"""C:\Users\pgustavo\Desktop\GruntHTTP.exe""",process--1f79fffc-7c81-5349-9f86-c1b170168dde,5520,cmd.exe,None,None,None,file--bdb40b46-65f7-5a74-9516-60746576d4e7,C:\Windows\System32,directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622,THESHIRE\pgustavo,user-account--2dfb4c1a-ebd5-5c42-a8f1-26acde5c50eb,THESHIRE\pgustavo,user-account--2dfb4c1a-ebd5-5c42-a8f1-26acde5c50eb
{b34bc01c-add5-5f66-2210-000000000400},2020-09-20T01:18:13.416Z,InstallUtil.exe,C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll,process--fb38481b-6881-53a8-9f2a-81afd9fbe548,None,InstallUtil.exe,None,None,None,file--b60b324c-e673-5ea0-b0f5-ce9daab709ef,C:\Windows\Microsoft.NET\Framework64\v4.0.30319,directory--5c954cca-a5bd-5cc5-821f-226ae6154055,{b34bc01c-add5-5f66-2010-000000000400},2020-09-20T01:18:13.287Z,cmd.exe,"""cmd.exe"" /c C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll",process--d689999d-1ba7-5310-8af6-78edc64d9b28,3776,{b34bc01c-0266-5f65-540c-000000000400},None,GruntHTTP.exe,"""C:\Users\pgustavo\Desktop\GruntHTTP.exe""",process--1f79fffc-7c81-5349-9f86-c1b170168dde,5520,cmd.exe,None,None,None,file--bdb40b46-65f7-5a74-9516-60746576d4e7,C:\Windows\System32,directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622,THESHIRE\pgustavo,user-account--2dfb4c1a-ebd5-5c42-a8f1-26acde5c50eb,THESHIRE\pgustavo,user-account--2dfb4c1a-ebd5-5c42-a8f1-26acde5c50eb
